In [14]:
import torch

In [15]:
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
from common.multiprocessing_env import SubprocVecEnv
from common.minipacman import MiniPacman
from common.rollout_encoder import RolloutEncoder
from common.imagination_core import ImaginationCore
from common.i2a import I2A
from common import pacman_util as putil
from common.environment_model import EnvModel
from common.actor_critic import ActorCritic
from common import torch_util as tutil
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [17]:
mode = 'regular'
env = MiniPacman(mode, 1000)
state_shape = env.observation_space.shape
num_actions = env.action_space.n
num_rewards = len(putil.task_rewards[mode])

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [18]:
full_rollout = True
env_model = EnvModel(state_shape, putil.num_pixels, num_rewards)
env_model.load_state_dict(torch.load('weights/env_model_regular'))

In [19]:
distil_policy = ActorCritic(state_shape, num_actions)

In [20]:
imagination = ImaginationCore(1, state_shape, num_actions, num_rewards, env_model, distil_policy, full_rollout=full_rollout)
actor_critic = I2A(state_shape, num_actions, num_rewards, 256, imagination, full_rollout=full_rollout)

In [21]:
if tutil.USE_CUDA:
    env_model = env_model.cuda()
    distil_policy = distil_policy.cuda()
    actor_critic = actor_critic.cuda()

In [22]:
actor_critic.load_state_dict(torch.load('weights/i2a_150000'))

In [86]:
import time 
import numpy as np

def displayImage(image, step, reward, imagined_states, imagined_rewards):
    clear_output(True)
    
    s = "step: " + str(step) + " reward: " + str(reward)
    plt.figure(figsize=(10,3))
    plt.subplot(141)
    plt.title(s)
    plt.imshow(image)
    
    show = [(142, 0), (143, 1), (144, 2)]
    
    for mplt_id, i in show:
        plt.subplot(mplt_id)
        plt.title("Imagined A: " + str(np.argmax(imagined_rewards[i])))
        plt.imshow(imagined_states[i])
    
    plt.show()
    time.sleep(0.1)

In [88]:
done = False
state = env.reset()
total_reward = 0
step   = 1

while not done:
    current_state = torch.FloatTensor(state).unsqueeze(0)
    if tutil.USE_CUDA:
        current_state = current_state.cuda()
        
    action = actor_critic.act(tutil.Variable(current_state))
    
    next_state, reward, done, _ = env.step(action.data)
    total_reward += reward
    state = next_state
    
    image = torch.FloatTensor(state).permute(1, 2, 0).cpu().numpy()
    
    imagined_state = actor_critic.imagined_state[0].permute(0, 2, 3, 1).cpu().numpy()
    imagined_reward = actor_critic.imagined_reward.cpu().numpy()[0]
    
    displayImage(image, step, total_reward, imagined_state, imagined_reward)
    step += 1

KeyboardInterrupt: 